# Simulação Monte Carlo com MCX (PMCX)

Este notebook mostra como configurar e executar uma simulação de transporte de luz usando o **Monte Carlo eXtreme (MCX)** via a interface Python `pmcx`.

MCX simula transporte de fótons em meios túrbidos com aceleração em GPU. Aqui montamos um volume cúbico simples com dois tecidos (fundo e inclusão central) e calculamos a distribuição de fluência de luz.

### Pré-requisitos de ambiente
Como o MCX usa CUDA para paralelização massiva, ative a GPU:
1. Vá em **Runtime**.
2. Selecione **Change runtime type**.
3. Em **Hardware accelerator**, escolha **GPU** (T4/L4/A100 ou similar).

## 1. Instalando dependências

Vamos instalar o pacote `pmcx`, que inclui os binários do MCX e utilitários para ler/gravar os formatos JSON/Binary usados pelo simulador.

In [ ]:
# Instala a interface Python do MCX e bibliotecas de visualização
!pip install -q pmcx numpy matplotlib jdata bjdata iso2mesh

## 2. Importando e checando a GPU

Antes de rodar a simulação, verificamos se a GPU NVIDIA foi detectada. O MCX precisa de GPU com CUDA.

A função `pmcx.gpuinfo()` lista os dispositivos disponíveis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pmcx

# Checar GPUs disponíveis
gpu_info = pmcx.gpuinfo()
print("GPUs encontradas:")
print(gpu_info)

if not gpu_info:
    print("AVISO: Nenhuma GPU NVIDIA detectada. O MCX pode não rodar ou ficará lento.")

## 3. Configuração do volume e da simulação

Aqui definimos o mundo físico da simulação e os parâmetros do MCX.

### A. Definindo o volume (`vol`)
- **Valor 1:** Tecido base.
- **Valor 2:** Inclusão central (ex.: tumor/heterogeneidade).
- **Valor 0:** Fundo/ar (fora do volume ou onde for 0).

### B. Dicionário de configuração (`cfg`)
`pmcx` usa um dicionário Python para passar parâmetros ao MCX. Campos-chave:
* `nphoton`: total de fótons (mais fótons → menos ruído, mais tempo).
* `tstart`, `tend`, `tstep`: janela temporal (CW usa 1 passo).
* `srcpos`, `srcdir`: posição/direção da fonte.
* `prop`: tabela de propriedades ópticas `[mua, mus, g, n]` por rótulo.
  * Linha 0: meio externo (Ar/Fundo).
  * Linha 1: voxels com valor 1.
  * Linha 2: voxels com valor 2.
* `autopilot`: deixa o MCX escolher threads/blocos ideais.

In [ ]:
# 1. Definindo o volume (grade 60x60x60)
vol = np.ones([60, 60, 60], dtype='uint8')

# Inclusão central (meio 2)
vol[20:40, 20:40, 20:40] = 2

# 2. Configuração da simulação
cfg = {
    'nphoton': 1e7,          # 10 milhões de fótons
    'vol': vol,              # Volume definido acima
    'tstart': 0,             # Início (s)
    'tend': 5e-9,            # Fim (5 ns)
    'tstep': 5e-9,           # Passo de tempo (CW)
    'srcpos': [30, 30, 0],   # Posição da fonte
    'srcdir': [0, 0, 1],     # Direção (para +z)

    # Propriedades ópticas [mua, mus, g, n]
    'prop': [
        [0, 0, 1, 1],             # Meio 0 (Fundo/Ar)
        [0.005, 1.0, 0.9, 1.37],  # Meio 1 (Tecido normal)
        [0.05, 2.0, 0.9, 1.37],   # Meio 2 (Inclusão)
    ],

    'issrcfrom0': 1,         # Origem em [0,0,0]
    'autopilot': 1,          # MCX escolhe threads/blocos
    'gpuid': 1               # Usa a primeira GPU disponível
}
print(cfg.keys())

## 4. Rodando a simulação
`pmcx.run(cfg)` traduz o dicionário, lança a simulação e retorna `res` com dados e metadados.

In [ ]:
# Executa a simulação
res = pmcx.run(cfg)

print("Simulação finalizada!")
print("Chaves retornadas:", res.keys())
print('Stat', res['stat'])

## 5. Visualizando resultados
O principal resultado está em `flux` (array 4D). Para visualizar, extraímos um corte Y=30, aplicamos log10 e plotamos.

## 6. Visualizando o volume simulado
Para interpretar a fluência, mostramos o volume (onde a inclusão está).

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(vol[:, 30, :].T, cmap='viridis', origin='upper')
plt.colorbar(label='Tipo de meio (1: Tecido, 2: Inclusão)')
plt.title('Volume Simulado (Corte Y=30)')
plt.xlabel('X (voxels)')
plt.ylabel('Z (profundidade)')
plt.show()

Compare o volume com o mapa de fluência: a inclusão altera a distribuição de luz.

In [ ]:
# Extrai o mapa de fluência
fluence = res['flux']
slice_y = np.log10(fluence[:, 30, :, 0])

plt.figure(figsize=(10, 5))
plt.imshow(slice_y.T, cmap='jet', origin='upper')
plt.colorbar(label='Fluência (log10 J/mm^2)')
plt.title('Distribuição de Luz (Corte Y=30)')
plt.xlabel('X (voxels)')
plt.ylabel('Z (profundidade)')
plt.show()

# 7. Comparando tipos de saída
MCX pode retornar Flux, Fluence e Energy via `outputtype`.

## Rodando a comparação
No próximo bloco rodamos três vezes mudando `outputtype` (Flux, Fluence, Energy) mantendo o mesmo volume e propriedades.

In [ ]:
# Configuração base
cfg_base = cfg.copy()
cfg_base['gpuid'] = 1
cfg_base['autopilot'] = 1

results = {}
types = ['flux', 'fluence', 'energy']

print("Iniciando comparação de saídas...")
for out_type in types:
    print(f"Simulando: {out_type}...")
    current_cfg = cfg_base.copy()
    current_cfg['outputtype'] = out_type
    res_temp = pmcx.run(current_cfg)
    data_4d = res_temp['flux']
    results[out_type] = np.sum(data_4d, axis=3)

print("Simulações concluídas.")

In [ ]:
# Plot comparativo (corte Y=30)
slice_idx = 30
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

im1 = axes[0].imshow(np.log10(results['flux'][:, slice_idx, :] + 1e-15).T, cmap='jet', origin='upper')
axes[0].set_title('Flux (Taxa de Fluência)\nLuz atravessando')
fig.colorbar(im1, ax=axes[0], label='log10(W/mm^2)')

im2 = axes[1].imshow(np.log10(results['fluence'][:, slice_idx, :] + 1e-15).T, cmap='jet', origin='upper')
axes[1].set_title('Fluence\nLuz acumulada no tempo')
fig.colorbar(im2, ax=axes[1], label='log10(J/mm^2)')

im3 = axes[2].imshow(np.log10(results['energy'][:, slice_idx, :] + 1e-15).T, cmap='jet', origin='upper')
axes[2].set_title('Energy Deposit\nOnde a luz é absorvida')
fig.colorbar(im3, ax=axes[2], label='log10(J/mm^3)')

plt.tight_layout()
plt.show()

center_flux = results['flux'][30, 30, 30]
center_energy = results['energy'][30, 30, 30]
print("--- Centro da inclusão ---")
print(f"Flux (luz disponível): {center_flux:.2e}")
print(f"Energy (luz absorvida): {center_energy:.2e}")
print("Energy destaca a inclusão pois mu_a é maior (0.05 vs 0.005).")

## Interpretando os resultados
Energy destaca a inclusão (maior absorção); Flux/Fluence mostram padrões de propagação e atenuação.

# 8. Boas práticas: reprodutibilidade e persistência
Guarde saídas em formato estável (.bnii) para refazer figuras sem rerrodar a física.

In [ ]:
import jdata

print("Salvando dados em disco...")
file_name = 'mcx_tumor_simulation.bnii'
jdata.save(results, file_name)
print(f"Arquivo {file_name} salvo!")

del results
print("Dados removidos da RAM.")

print("Carregando dados do arquivo...")
loaded_data = jdata.load(file_name)
print("Chaves recuperadas:", loaded_data.keys())

energy_map = loaded_data['energy']
slice_idx = 30
plt.figure(figsize=(6, 5))
plt.imshow(np.log10(energy_map[:, slice_idx, :] + 1e-15).T, cmap='jet', origin='upper')
plt.title('Dado carregado (.bnii)\nEnergia absorvida')
plt.colorbar(label='log10(J/mm^3)')
plt.show()

### Detalhes de formato
Usamos **BJData (Binary JData)** com extensão `.bnii`, parte da iniciativa NeuroJSON, eficiente para volumes grandes.

# 9. Modelagem avançada com `Shapes`
Podemos descrever geometrias em JSON (Shapes) em vez de montar `vol` manualmente.

Vantagens: simplicidade, escala com resolução e legibilidade.
Chaves novas no `cfg`: `dim` (tamanho da grade) e `shapes` (lista de primitivas).
Shapes são processados em ordem; o último sobrescreve os anteriores (Painter).
Comandos comuns: `Grid`, `Sphere`, `Box`, `ZLayers`, `Cylinder`.

In [ ]:
import jdata as jd

cfg_shapes = {
    'nphoton': int(1e7),
    'vol': np.zeros((60, 60, 60), dtype='uint8'),
    'tstart': 0.0,
    'tend': 5e-9,
    'tstep': 5e-9,
    'srcpos': [30, 30, 0],
    'srcdir': [0, 0, 1],
    'prop': [
        [0,    0,   1,   1],     # Tag 0: Ar
        [0.1, 10.0, 0.9, 1.4],   # Tag 1: Epiderme
        [0.01, 2.0, 0.9, 1.37],  # Tag 2: Derme
        [0.05, 5.0, 0.9, 1.37],  # Tag 3: Tumor
    ],
    'dumpmask': 1,
    'autopilot': 1,
    'gpuid': 1,
    'outputtype': 'fluence',
}

shape = []
shape.append({'Grid': {'Tag': 0, 'Size': [60, 60, 60]}})
shape.append({'ZLayers': [[0, 10, 1], [11, 60, 2]]})
shape.append({'Sphere': {'O': [30, 30, 30], 'R': 8, 'Tag': 3}})
cfg_shapes['shapes'] = jd.show({'Shapes': shape}, {'string': True})

print("Rodando simulação com Shapes...")
res_gpu = pmcx.run(cfg_shapes)
print("Simulação finalizada!")
print("Chaves disponíveis:", list(res_gpu.keys()))
if 'vol' in res_gpu:
    print("Volume retornado disponível em res['vol'].")
else:
    print("Aviso: 'vol' não encontrado no resultado.")

**Visualizando a geometria complexa**
Shapes gera a geometria; vamos visualizar o volume retornado e a fluência.

In [ ]:
real_gpu_vol = res_gpu['vol']
real_fluence = res_gpu['flux']

slice_idx = 30
img_vol_gpu = real_gpu_vol[:, slice_idx, :].T
img_flu_gpu = np.log10(np.sum(real_fluence, axis=3)[:, slice_idx, :] + 1e-15).T

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
im1 = axes[0].imshow(img_vol_gpu, origin='upper')
axes[0].set_title("Volume retornado (res['vol'])")
axes[0].set_xlabel('X (voxels)')
axes[0].set_ylabel('Z (profundidade)')
cbar1 = fig.colorbar(im1, ax=axes[0], ticks=[0, 1, 2, 3])
cbar1.ax.set_yticklabels(['Ar', 'Epiderme', 'Derme', 'Tumor'])

im2 = axes[1].imshow(img_flu_gpu, cmap='jet', origin='upper')
axes[1].set_title('Resultado da simulação')
axes[1].set_xlabel('X (voxels)')
axes[1].set_ylabel('Z (profundidade)')
fig.colorbar(im2, ax=axes[1], label='log10(Fluence)')

plt.tight_layout()
plt.show()

print("Geometria confirmada pelo volume retornado.")